In [1]:
import ngmix
import fitsio
import proplot as pplt
import numpy as np
import piff
import galsim
import yaml

In [2]:
%matplotlib inline

In [8]:
d = np.hstack([
    fitsio.read("files_000001.fits", lower=True),
    fitsio.read("files_000002.fits", lower=True),
])

msk = (
    (d["band"] == "g") | (d["band"] == "r") | (d["band"] == "i") | (d["band"] == "z")
)
d = d[msk]

import os


fnames = np.array(sorted([
    os.path.join(d["path"][i], d["filename"][i])
    for i in range(len(d))
]))

rng = np.random.RandomState(seed=10)
rng.shuffle(fnames)

In [ ]:
from des_y6utils.piff import (
    measure_star_t_for_piff_model,
    map_star_t_to_grid,
    measure_t_grid_for_piff_model,
    make_good_regions_for_piff_model_gal_grid,
    make_good_regions_for_piff_model_star_and_gal_grid,
    nanmad,
)
import tqdm
import joblib
import subprocess

os.system("mkdir -p ./data")

def _download_file(fname):
    if not os.path.exists(os.path.join(os.environ["DESDATA"], fname)):
        cmd = "mkdir -p " + os.path.dirname(os.path.join(os.environ["DESDATA"], fname))
        subprocess.run(cmd, shell=True)
        cmd = r"""\
rsync \
            -av \
            --password-file $DES_RSYNC_PASSFILE \
            ${DESREMOTE_RSYNC_USER}@${DESREMOTE_RSYNC}/%(fname)s \
            %(source_dir)s/%(fname)s
""" % dict(fname=fname, source_dir=os.environ["DESDATA"])
        res = subprocess.run(cmd, shell=True, capture_output=True, check=True)
        if res.returncode != 0:
            print(res.stdout.decode("utf-8"), res.stderr.decode("utf-8"), flush=True)


def _process(fname):
    try:
        _download_file(fname)

        piff_mod = piff.read(os.path.join(os.environ["DESDATA"], fname))

        if "_z_" in fname:
            piff_kwargs = {"IZ_COLOR": 0.34}
        else:
            piff_kwargs = {"GI_COLOR": 0.61}

        t_arr = measure_t_grid_for_piff_model(piff_mod, piff_kwargs, seed=14354, grid_size=128)
        return t_arr.ravel(), os.path.basename(fname)
    except Exception:
        return None
    
        

chunksize = 128
nchunks = len(fnames) // 128 + 1
    
loc = 0
for chunk in tqdm.trange(nchunks):
    if os.path.exists("./data/data_chunk%06d.fits" % chunk):
        continue
    start = chunk * chunksize
    stop = min(start + chunksize, len(fnames))
    
    jobs = []
    for fname in fnames[start:stop]:
        jobs.append(joblib.delayed(_process)(fname))

    with joblib.Parallel(n_jobs=-1, verbose=0) as par:
        output = par(jobs)


    arrs = [out[0] for out in output if out is not None]
    arr_fnames = [out[1] for out in output if out is not None]
    arr = np.hstack(arrs)

    fitsio.write("./data/data_chunk%06d.fits" % chunk, arr, clobber=True)
    fitsio.write("./data/fnames_chunk%06d.fits" % chunk, {"fnames": np.array(arr_fnames)}, clobber=True)

  2%|██▏                                                                                                                             | 142/8437 [01:26<1:52:45,  1.23it/s]/gpfs02/astro/workarea/beckermr/miniconda3/envs/desy6/lib/python3.9/site-packages/ngmix/admom/admom.py:401: RuntimeWarning: divide by zero encountered in double_scalars
  res['flux_mean'] = flux_sum/res['wsum']
  2%|██▏                                                                                                                             | 143/8437 [02:32<3:51:40,  1.68s/it]/gpfs02/astro/workarea/beckermr/miniconda3/envs/desy6/lib/python3.9/site-packages/ngmix/admom/admom.py:401: RuntimeWarning: divide by zero encountered in double_scalars
  res['flux_mean'] = flux_sum/res['wsum']
  2%|██▏                                                                                                                             | 144/8437 [03:38<6:37:35,  2.88s/it]/gpfs02/astro/workarea/beckermr/miniconda3/envs/desy6/lib/python3.9/